In [6]:
import cv2
import numpy as np
import os
from os import listdir
from os.path import isfile, join

In [1]:


mypath = r'\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for read_path in onlyfiles:
    final_path = join(mypath, read_path)
    print(f"Processing {final_path}")  # Debugging line to print the file path being processed

    img = cv2.imread(final_path, cv2.IMREAD_COLOR)

    if img is None:
        print(f"Failed to load image at {final_path}")
        continue
    blurred_img = cv2.GaussianBlur(img, (3, 3), 0)
    hsv = cv2.cvtColor(blurred_img, cv2.COLOR_BGR2HSV)

    # Define the range of blue color in HSV
    low_blue = np.array([55, 50, 50])
    high_blue = np.array([130, 255, 255])

    # Define the range of red color in HSV
    low_red = np.array([130, 50, 50])
    high_red = np.array([180, 255, 255])

    # Define HSV range for brown color
    low_brown = np.array([0, 50, 40])
    high_brown = np.array([22, 100, 240])

    # Create masks based on the color ranges
    blue_mask = cv2.inRange(hsv, low_blue, high_blue)
    red_mask = cv2.inRange(hsv, low_red, high_red)
    brown_mask = cv2.inRange(hsv, low_brown, high_brown)

    # Create a blank image with the same dimensions as the original image
    overlay = np.zeros_like(img)

    # Assign colors to the overlay based on masks
    overlay[blue_mask != 0] = [255, 0, 0]  # Blue mask as red (BGR format)
    overlay[red_mask != 0] = [0, 0, 255]  # Red mask as blue (BGR format)
    overlay[brown_mask != 0] = [0, 255, 0]  # Brown mask as custom color (BGR format)

    # Combine the overlay with the original image
    masked_image = cv2.addWeighted(img, 0.7, overlay, 0.3, 0)
    out_path = r'\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\attempt1'

    if not os.path.exists(out_path):
        os.makedirs(out_path)

    full_path = join(out_path, read_path)
    cv2.imwrite(full_path, masked_image)
    print(f"Saved processed image to {full_path}")  # Debugging line to confirm save

Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\2x\6521-001_HE.tif


KeyboardInterrupt: 

In [8]:
def mouseRGB(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  # Checks mouse left button down condition
        colorsB = image[y, x, 0]
        colorsG = image[y, x, 1]
        colorsR = image[y, x, 2]
        colors = image[y, x]
        hsv_value = np.uint8([[[colorsB, colorsG, colorsR]]])
        hsv = cv2.cvtColor(hsv_value, cv2.COLOR_BGR2HSV)
        print("HSV :", hsv)
        print("Red: ", colorsR)
        print("Green: ", colorsG)
        print("Blue: ", colorsB)
        print("BRG Format: ", colors)
        print("Coordinates of pixel: X: ", x, "Y: ", y)

# Read an image
image_path = r"\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\2x_v2\chop\6521-004_IHCC"
image = cv2.imread(image_path, cv2.IMREAD_COLOR)

# Verify image loading
if image is None:
    print(f"Failed to load image at {image_path}")
else:
    # Print image dimensions
    print(f"Image loaded with dimensions: {image.shape}")

    # Create a window and bind the function to window
    cv2.namedWindow('mouseRGB', cv2.WINDOW_NORMAL)
    cv2.setMouseCallback('mouseRGB', mouseRGB)

    # Do until esc pressed
    while True:
        cv2.imshow('mouseRGB', image)
        if cv2.waitKey(20) & 0xFF == 27:  # ESC key to break
            break

    # If esc pressed, finish
    cv2.destroyAllWindows()

Failed to load image at \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\2x_v2\chop\6521-004_IHCC


In [3]:
import cv2
import os
import numpy as np
from os import listdir
from os.path import isfile, join

# Initialize global variables
zoom_factor = 1.0
pan_x, pan_y = 0, 0
cursor_x, cursor_y = 0, 0
hsv_values = []  # List to store HSV values

def mouseRGB(event, x, y, flags, param):
    global zoom_factor, pan_x, pan_y, cursor_x, cursor_y, hsv_values

    if event == cv2.EVENT_LBUTTONDOWN:  # Check mouse left button down condition
        colorsB = display_img[y, x, 0]
        colorsG = display_img[y, x, 1]
        colorsR = display_img[y, x, 2]
        colors = display_img[y, x]
        hsv_value = np.uint8([[[colorsB, colorsG, colorsR]]])
        hsv = cv2.cvtColor(hsv_value, cv2.COLOR_BGR2HSV)
        print("Left Click - HSV :", hsv)
        print("Red: ", colorsR)
        print("Green: ", colorsG)
        print("Blue: ", colorsB)
        print("BRG Format: ", colors)
        print("Coordinates of pixel: X: ", x, "Y: ", y)
        mouseRGB.last_x, mouseRGB.last_y = x, y

    elif event == cv2.EVENT_RBUTTONDOWN:  # Check mouse right button down condition
        colorsB = display_img[y, x, 0]
        colorsG = display_img[y, x, 1]
        colorsR = display_img[y, x, 2]
        colors = display_img[y, x]
        hsv_value = np.uint8([[[colorsB, colorsG, colorsR]]])
        hsv = cv2.cvtColor(hsv_value, cv2.COLOR_BGR2HSV)
        print("Right Click - HSV :", hsv)
        print("Red: ", colorsR)
        print("Green: ", colorsG)
        print("Blue: ", colorsB)
        print("BRG Format: ", colors)
        print("Coordinates of pixel: X: ", x, "Y: ", y)
        mouseRGB.last_x, mouseRGB.last_y = x, y

        # Append HSV value to the list
        hsv_values.append(hsv.tolist())

    elif event == cv2.EVENT_MOUSEWHEEL:
        cursor_x, cursor_y = x, y
        if flags > 0:
            zoom_factor += 0.1
        else:
            zoom_factor -= 0.1
        zoom_factor = max(0.1, zoom_factor)
        update_display_img()

    elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
        pan_x += x - mouseRGB.last_x
        pan_y += y - mouseRGB.last_y
        mouseRGB.last_x, mouseRGB.last_y = x, y
        update_display_img()

def update_display_img():
    global display_img, zoom_factor, pan_x, pan_y, cursor_x, cursor_y
    h, w = img.shape[:2]
    # Center of the zoom
    center_x, center_y = cursor_x, cursor_y
    M = np.float32([[zoom_factor, 0, center_x - center_x * zoom_factor + pan_x],
                    [0, zoom_factor, center_y - center_y * zoom_factor + pan_y]])
    display_img = cv2.warpAffine(img, M, (w, h))

mypath = r'\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\2x_v2\chop'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

zoom_factor = 1.0
pan_x, pan_y = 0, 0
cursor_x, cursor_y = 0, 0
mouseRGB.last_x = 0
mouseRGB.last_y = 0

for read_path in onlyfiles:
    final_path = join(mypath, read_path)
    print(f"Processing {final_path}")  # Debugging line to print the file path being processed

    img = cv2.imread(final_path, cv2.IMREAD_COLOR)

    if img is None:
        print(f"Failed to load image at {final_path}")
        continue

    display_img = img.copy()

    cv2.namedWindow('mouseRGB', cv2.WINDOW_NORMAL)
    cv2.setMouseCallback('mouseRGB', mouseRGB)

    while True:
        cv2.imshow('mouseRGB', display_img)
        key = cv2.waitKey(20) & 0xFF
        if key == 27:  # ESC key to break
            break
        elif key == ord('x'):  # Press 'x' key to break the for loop
            break

    if key == ord('x'):  # Check if 'x' was pressed
        break  # Break out of the for loop

    cv2.destroyAllWindows()

# Convert the nested list to a NumPy array for easier manipulation
array = np.array(hsv_values)

# Find the maximum value in each column (axis 0)
max_values = np.max(array, axis=0)

# Convert the result to a list
max_values_list = max_values.tolist()

# Find the maximum value in each column (axis 0)
min_values = np.min(array, axis=0)

# Convert the result to a list
min_values_list = min_values.tolist()
print(min_values_list)
print(max_values_list)

Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\2x_v2\chop\6521-001_HE.tif
Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\2x_v2\chop\6521-003_IHCB.tif
Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\2x_v2\chop\6521-004_IHCC.tif
Left Click - HSV : [[[120   1 237]]]
Red:  236
Green:  236
Blue:  237
BRG Format:  [237 236 236]
Coordinates of pixel: X:  1693 Y:  2940
Left Click - HSV : [[[  0   0 245]]]
Red:  245
Green:  245
Blue:  245
BRG Format:  [245 245 245]
Coordinates of pixel: X:  4227 Y:  2788
Left Click - HSV : [[[ 20   7 210]]]
Red:  210
Green:  208
Blue:  204
BRG Format:  [204 208 210]
Coordinates of pixel: X:  3018 Y:  2737
Left Click - HSV : [[[150   1 238]]]
Red:  238
Green:  237
Blue:  238
BRG Format:  [238 237 238]
Coordinates of pixel: X:  1700 Y:  3021
Left Click - HSV : [[[123  10 220]]]
Red:  212
Green:  211
Blue:  220
BRG Format:  [220 211 212]
Coordinates of pixel: X:  1827 Y:  2748
Left Click - H

In [ ]:
mypath = r'\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop'
for read_path in onlyfiles:
    final_path = join(mypath, read_path)
    print(f"Processing {final_path}")  # Debugging line to print the file path being processed

    img = cv2.imread(final_path, cv2.IMREAD_COLOR)

    if img is None:
        print(f"Failed to load image at {final_path}")
        continue

    blurred_img = cv2.GaussianBlur(img, (3, 3), 0)
    hsv = cv2.cvtColor(blurred_img, cv2.COLOR_BGR2HSV)

    # Define the range of blue color in HSV
    #low_blue = np.array([55, 50, 50])
    #high_blue = np.array([130, 255, 255])

    # Define the range of red color in HSV
    #low_red = np.array([130, 50, 50])
    #high_red = np.array([180, 255, 255])

    # Define HSV range for brown color
    #low_brown = np.array([0, 50, 40])
    #high_brown = np.array([22, 100, 240])
    low_brown = np.asarray(min_values_list[0][0])
    high_brown = np.asarray(max_values_list[0][0])

    # Create masks based on the color ranges
    #blue_mask = cv2.inRange(hsv, low_blue, high_blue)
    #red_mask = cv2.inRange(hsv, low_red, high_red)
    brown_mask = cv2.inRange(hsv, low_brown, high_brown)

    # Create a blank image with the same dimensions as the original image
    overlay = np.zeros_like(img)

    # Assign colors to the overlay based on masks
    #overlay[blue_mask != 0] = [255, 0, 0]  # Blue mask as red (BGR format)
    #overlay[red_mask != 0] = [0, 0, 255]  # Red mask as blue (BGR format)
    overlay[brown_mask != 0] = [0, 255, 0]  # Brown mask as custom color (BGR format)

    # Combine the overlay with the original image
    masked_image = cv2.addWeighted(img, 0.7, overlay, 0.3, 0)
    out_path = r'\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\attempt1'

    if not os.path.exists(out_path):
        os.makedirs(out_path)

    full_path = join(out_path, read_path)
    cv2.imwrite(full_path, masked_image)
    print(f"Saved processed image to {full_path}")  # Debugging line to confirm save"""

Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\6521-001_HE.tif
Failed to load image at \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\6521-001_HE.tif
Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\6521-003_IHCB.tif
Saved processed image to \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\attempt1\6521-003_IHCB.tif
Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\6521-004_IHCC.tif
Saved processed image to \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\attempt1\6521-004_IHCC.tif
Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\6521-005_HE.tif
Failed to load image at \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\6521-005_HE.tif
Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\6521-007_IHCB.tif
Saved processed image to \\10.17.97.73\kie

In [3]:
mypath = r'\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\2x_v2\chop'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for read_path in onlyfiles:
    final_path = join(mypath, read_path)
    print(f"Processing {final_path}")  # Debugging line to print the file path being processed

    img = cv2.imread(final_path, cv2.IMREAD_COLOR)
    cv2.namedWindow('mouseRGB')
    cv2.setMouseCallback('mouseRGB', mouseRGB)

    # Display the image in the window
    while True:
        cv2.imshow('mouseRGB', img)
        if cv2.waitKey(20) & 0xFF == 27:  # ESC key to break
            break

    # Close the window after ESC key is pressed
    cv2.destroyAllWindows()

Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\2x_v2\chop\6521-001_HE.tif
Left Click - HSV : [[[111   8 211]]]
Red:  204
Green:  206
Blue:  211
BRG Format:  [211 206 204]
Coordinates of pixel: X:  3204 Y:  1757
Right Click - HSV : [[[150  41 198]]]
Red:  198
Green:  166
Blue:  198
BRG Format:  [198 166 198]
Coordinates of pixel: X:  3370 Y:  1736
Right Click - HSV : [[[155  72 159]]]
Red:  159
Green:  114
Blue:  152
BRG Format:  [152 114 159]
Coordinates of pixel: X:  3021 Y:  1703
Right Click - HSV : [[[149  76 161]]]
Red:  159
Green:  113
Blue:  161
BRG Format:  [161 113 159]
Coordinates of pixel: X:  2573 Y:  1328
Right Click - HSV : [[[162  32 221]]]
Red:  221
Green:  193
Blue:  210
BRG Format:  [210 193 221]
Coordinates of pixel: X:  2160 Y:  1195
Right Click - HSV : [[[137  86 140]]]
Red:  120
Green:  93
Blue:  140
BRG Format:  [140  93 120]
Coordinates of pixel: X:  2133 Y:  1682
Left Click - HSV : [[[154  61 171]]]
Red:  171
Green:  130
Blue:  165
BRG F

KeyboardInterrupt: 